# <font style="color: Red">Initialize & Login</font>

## <font style="color: green">Intialize</font>

In [ ]:
# run nothing because of bug of VS Code, this will error out
$env:PSModulePath.Split([system.IO.Path]::PathSeparator)[0]
$PSDefaultParameterValues

"`npid: $pid"

In [ ]:
$timeframe_to_investigate = 180 # in days

$clientID = '3fbb68ad-a29b-4c79-a709-8922e1a672f0' # Enterprise Application Client ID
$tenantId = 'cc140c86-814b-4468-b4f4-b4147ab5fe25' # Tenant ID

$look_back_date = Get-Date ((Get-Date).AddDays( $( 0 - $timeframe_to_investigate) )) -Format 'yyyy-MM-dd'

## <font style="color: green">Sign-in into the tenant</font>

We can use any account which is present in the tenant and had been assigned to the enterprise application

If this account is a Guest and MFA is enforced, we'll need to sign-in interactively and complete the MFA requirements

In [ ]:
# Disconnect-Graph | Out-Null
Connect-Defender -TenantId $tenantId -ClientID $clientID -PassThru | Format-List

# <font style="color: red">Incident enrichment</font>

## <font style="color: green">Get incidents</font>

In [ ]:
$incidents = $null
$incidents = try { Get-MgSecurityIncident -ExpandProperty Alerts -Filter "Status ne 'resolved' and Severity in ('medium', 'high') and CreatedDateTime ge $look_back_date" -ErrorAction Stop } catch { Write-Error "Failed to retrieve incidents: $_"; $error.Clear(); $null }
Write-Host "Incident count: $($incidents.count)"

## <font style="color: green">Get incident details from alerts</font>

### <font style="color: blue">Enrich with alert information</font>

In [ ]:
Write-Output "Incident count: $($incidents.Count)"

$swIncidents = [System.Collections.Specialized.OrderedDictionary]::new()

# iterate over the incidents (TEMPORARY ONLY SELECTING THE FIRST)
foreach ( $incident in ($incidents <#| Where-Object {$_.AssignedTo -eq $me.mail}#> )) {
    # create and build our "switch-incident" object
    $swIncident = @{}
    $swIncident.incId = $incident.Id
    $swIncident.incSev = $incident.Severity
    $swIncident.incTitle = $incident.DisplayName
    $swIncident.AlertCount = $incident.Alerts.Count
    $swIncident.Created = $incident.CreatedDateTime
    $swIncident.CustomTags = $incident.customTags
    $swIncident.AssignedTo = $incident.assignedTo

    # get the alert(s) of the indcident (THIS WAS NOT TESTED WITH MULTIPLE ALERTS in one incident)
    $alerts = foreach ($aId in $incident.Alerts.id) { Get-MgSecurityAlertV2 -AlertId $aId }
    Write-Output "Alert count: $($alerts.Count)"

    # iterate over the alerts
    foreach ($alert in $alerts) {

        # extent our incident object further
        $swIncident.AlertTitle = $alert.Title

        # different logic depeding on the DetectorId in question
        switch ($alert.DetectorId) {
            'b8f6b088-5487-4c70-037c-08d8d71a43fe' {<# Email messages removed after delivery#>}
            default {
                #Write-Output "Evidence count: $($alert.Evidence.Count)"
                Foreach($ev in $alert.Evidence){
                    $evOb = [pscustomobject][hashtable]($ev.AdditionalProperties)
                    switch ($evOb.'@odata.type') {
                        "#microsoft.graph.security.userEvidence" { $swIncident.dUPN = $evOb.userAccount.userPrincipalName;
                            $swIncident.AccountObjectId = $evOb.userAccount.azureAdUserId;}
                        "#microsoft.graph.security.ipEvidence" { $swIncident.dIP = $evOb.IPAddress }
                        "#microsoft.graph.security.cloudLogonRequestEvidence" { $swIncident.dRequestId = $evOb.requestId}
                        "#microsoft.graph.security.cloudApplicationEvidence" { $swIncident.dSource = $evOb.displayName }
                        "#microsoft.graph.security.fileEvidence" { $swIncident.fileSHA256 = $evOb.fileDetails.sha256; $swIncident.fileName = $evOb.fileDetails.fileName }
                        "#microsoft.graph.security.urlEvidence" { $swIncident.dURL = $evOb.url.replace(":","_")}
                        "#microsoft.graph.security.deviceEvidence" { $swIncident.mdeID = $evOb.mdeDeviceId; $swIncident.device = $evOb.deviceDnsName}
                        "#microsoft.graph.security.mailboxEvidence" {  
                            $swIncident.MB_primaryAddress = $evOb.primaryAddress;
                        }
                        "#microsoft.graph.security.analyzedMessageEvidence" {  
                            $swIncident.ME_deliveryLocation = $evOb.deliveryLocation;
                            $swIncident.ME_deliveryAction = $evOb.deliveryAction;
                            $swIncident.ME_Verdict = $ev.Verdict; 
                            $swIncident.ME_threats = $evOb.threats;
                            $swIncident.ME_CreatedDateTime = $evOb.CreatedDateTime; 
                            $swIncident.ME_receivedDateTime = $evOb.receivedDateTime;
                            $swIncident.ME_senderIp = $evOb.senderIp;
                            $swIncident.ME_subject = $evOb.subject;
                            $swIncident.ME_networkMessageId = $evOb.networkMessageId;
                            $swIncident.dURL = $evOb.urls | foreach-object { $_.replace(":","_") };
                        }


                        "#microsoft.graph.security.processEvidence" {}
                        
                        
                        "#microsoft.graph.security.mailClusterEvidence" { <# This type can have many #>}
                        default { "$($alert.DetectorId)`t$($evOb.'@odata.type')" }
                    }

                }
            }
        }
        if ($swIncident.dUPN) {
            $sortUPN = $swIncident.dUPN.Split('@')[0]
        
            $user = Get-MgUser -Filter "userPrincipalName eq '$($swIncident.dUPN)'" -Select OnPremisesExtensionAttributes, Id, DisplayName, mail, userPrincipalName, EmployeeType
            if ($user) {
        
                if ($user.OnPremisesExtensionAttributes.ExtensionAttribute1){
                    if ( $user.OnPremisesExtensionAttributes.ExtensionAttribute1 -eq 'H' ) { $swIncident.dAccountType = "Faculty" }
                    elseif ( $user.OnPremisesExtensionAttributes.ExtensionAttribute1 -eq 'C' ) { $swIncident.dAccountType = "Student" }
                    elseif ( $( try { $user.OnPremisesExtensionAttributes.ExtensionAttribute1.Contains(',') } catch {$false} )  ) {$swIncident.dAccountType = "Faculty, Student"}
                } else { 
                    if ( $user.EmployeeType -eq 'H' ) { $swIncident.dAccountType = "Faculty" }
                    elseif ( $user.EmployeeType -eq 'C' ) { $swIncident.dAccountType = "Student" }
                    elseif ( ((Invoke-GraphRequest -Uri "/v1.0/users/$($user.Id)/memberOf" -Method Get).Value.onPremisesSamAccountName -match "for_faculty").count -gt 0 ) { $swIncident.dAccountType = "Faculty" }
                    else { Write-Warning "$($swIncident.dUPN) has no ExtensionAttribute1" }
                }
                #if ( $sortUPN.length -eq 4 ) { $swIncident.dAccountType = "Faculty" }
                #elseif ( ($sortUPN.length -eq 6 -or $sortUPN.length -eq 7) -and $(try{ [int]$sortUPN.Substring(5) -is [int] } catch {$false}) ) { $swIncident.dAccountType = "Student" }
                #elseif ( $sortUPN.startswith('adm-')) { $swIncident.dAccountType = "Admin" }
                #else { $swIncident.dAccountType = "ServiceAccount" }
                
                $swIncident.dMail = $user.mail
            } else {
                $user = Get-MgUser -Filter "mail eq '$($swIncident.dUPN)'" -Select OnPremisesExtensionAttributes, Id, DisplayName, mail, userPrincipalName, EmployeeType
                
                if ($user) {
                    if ( $sortUPN.length -eq 4 ) { $swIncident.dAccountType = "Faculty" }
                    elseif ( ($sortUPN.length -eq 6 -or $sortUPN.length -eq 7) -and $(try{ [int]$sortUPN.Substring(5) -is [int] } catch {$false}) ) { $swIncident.dAccountType = "Student" }
                    elseif ( $sortUPN.startswith('adm-')) { $swIncident.dAccountType = "Admin" }
                    elseif ( $user.UserPrincipalName.contains('#EXT#@')) { $swIncident.dAccountType = "Guest" }
                    
                    $swIncident.dMail = $user.mail
                }
            }
        }
    }
    $swIncidents.Add($swIncident.incId,[pscustomobject]$swIncident)
}

### <font style="color: blue">Enrich with IP information</font>

> As `Get-IPRiskDetails` is not implemented here we can't run the next block. Hence the `while ($false)` block

In [ ]:
while ($false){
    ## enrich IPAddress
    if ($swIncidents.count -gt 0) {
        foreach( $incident in $swIncidents.Values.GetEnumerator() ) {
            $ipx = ''
            $vpnProvider = ''
            if ( $incident.dIP ) {
                $ipDetails = Get-IPRiskDetails $incident.dIP
                if (Test-KnownVPN $ipDetails) {$vpnProvider = $ipDetails.organization.Replace(" ",'')}
                if ($ipDetails.proxy -eq 'True') { $ipx += "P" } else { $ipx += "." }
                if ($ipDetails.tor -eq 'True') { $ipx += "T" } else { $ipx += "." }
                if ($ipDetails.vpn -eq 'True') { $ipx += "V" } else { $ipx += "." }
            }
            
            try { Add-Member -InputObject $incident -Name "xIP" -Value $ipx -MemberType NoteProperty -Force }
            catch {}
            try { Add-Member -InputObject $incident -Name "xVPN" -Value $vpnProvider -MemberType NoteProperty -Force }
            catch {}
        }
    } else {
        Write-Warning "No incidents found"
    }
}

In [ ]:
# Print incident details
# $swIncidents.Values.GetEnumerator() | Format-list  incId, ME_Verdict, dAccountType, dIP, xIP, incTitle
 $swIncidents.Values.GetEnumerator() | Format-table incId, ME_Verdict, dAccountType, dIP, xIP, incTitle

### <font style="color: blue">Tag the incidents</font>

In [ ]:

Set-incidentTags

# <font style="color: Red">Process incidents</font>

Call nested notebooks

## <font style="color: green">Convert incidents</font>

In [ ]:
$me = Invoke-MgGraphRequest -Method GET -Uri /v1.0/me | Select-Object -Property id, displayName, userPrincipalName, mail
$converted = ($swIncidents.Values.GetEnumerator()) | ConvertTo-Json

## <font style="color: green">Call nested notebooks</font>

### <font style="color: blue">Anomalous Token involving one user (Student)</font>

In [ ]:
$nested_anonTokenNotebook = Join-Path $pwd 'nested-notebooks' 'exclude-all-anonym-ip.ipynb'
Invoke-ExecuteNotebook $nested_anonTokenNotebook -Parameters @{arr = $converted; me = $me}

### <font style="color: blue">Anonymous IP address involving one user (Student)</font>

#### <font style="color:red">Not implemented</font>

In [ ]:
$nested_anonTokenNotebook = Join-Path $pwd 'nested-notebooks' 'exclude-anonym-ip.ipynb'
Invoke-ExecuteNotebook $nested_anonTokenNotebook -Parameters @{arr = $converted; me = $me}

### <font style="color: blue">Activity from an anonymous proxy involving one user (Student)</font>

#### <font style="color:red">Not implemented</font>

In [ ]:
$nested_anonTokenNotebook = Join-Path $pwd 'nested-notebooks' 'exclude-anonym-proxy.ipynb'
Invoke-ExecuteNotebook $nested_anonTokenNotebook -Parameters @{arr = $converted; me = $me}